In [ ]:
# Plot various data

import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import math

figsize=(4.5, 3)

In [ ]:
# Only read this one CSV.
# Change this, if you are interested in different ciphers
data = pd.read_csv("DHE-RSA-AES128-GCM-SHA256.csv")

In [ ]:
data1C = data[data['numConns'] == 1]
#data1C

In [ ]:
data1C.loc[[0]]['dataLen'] = 500
#data1C

In [ ]:
# Normalize the data
for i,r in data1C.iterrows():
    r['openssl'] /= r['numBytes']
    r['denseMap'] /= r['numPkts']
    r['tbb'] /= r['numPkts']
    r['siphash'] /= r['numPkts']
    

In [ ]:
#plt.plot(data1C['dataLen'], data1C['openssl'])

# Plot the cost of siphash
fig = plt.figure(figsize=figsize, dpi=300)
plt.plot(data1C['dataLen'], data1C['siphash'])
plt.semilogx()
plt.xlabel('Data Length')
plt.ylabel('Cycles')
plt.title("Cost of siphash by data length")

plt.grid(b=True)

plt.savefig('output/siphash.pdf',bbox_inches='tight')
plt.show()

In [ ]:
# Plot the cost of TBB's shared map
tbbMean = data.groupby(['numConns'])['tbb'].mean()
steps = data['numConns'].unique()

[print("{" + str(x) + "," + str(tbbMean[x]) + "},", end='') for x in data['numConns'].unique()]

fig = plt.figure(figsize=figsize, dpi=300)
plt.plot(steps, tbbMean)
plt.ylim([0,40000])
plt.xlabel('#Connections')
plt.ylabel('Cycles')
plt.title("Cost of shared map by number of connections")

plt.grid(b=True)

plt.savefig('output/tbb.pdf',bbox_inches='tight')
plt.show()

In [ ]:
# Plot the cost of the memory allocation
memMean = data.groupby(['numConns'])['memory'].mean()

#print(memMean)

memMeanPC = [memMean[x] / x for x in data['numConns'].unique()]

#[print("{" + str(x) + "," + str(memMean[x]) + "},", end='') for x in data['numConns'].unique()]

fig = plt.figure(figsize=figsize, dpi=300)
plt.plot(steps, memMean)
plt.ylim([0,15000])
plt.xlabel('#Connections')
plt.ylabel('Cycles')
plt.title("Cost of memory allocation by number of connections")

plt.grid(b=True)

plt.savefig('output/memory.pdf',bbox_inches='tight')
plt.show()

In [ ]:
# Build logarithmic data
dataDense = data
for _,r in dataDense.iterrows():
    r['denseMap'] /= r['numPkts']
    r['dataLen'] = math.log10(r['dataLen'])
    r['numConns'] = math.log(r['numConns'],2)

In [ ]:
# Plot the denseMap cost in 3D

denseMapMean = dataDense.groupby(['numConns'])['denseMap'].mean()
steps = data['numConns'].unique()

import numpy as np

#X = dataDense['dataLen'].apply(math.log10)
X = [np.arange(2,8)] * 6

#Y = dataDense['numConns'].apply(lambda x: math.log(x,2))
Y = [[x] * 6 for x in range(1,7)]

#Z = dataDense['denseMap']
Z = dataDense.pivot(index='numConns', columns='dataLen', values='denseMap').as_matrix()

#fig = plt.figure(figsize=(16, 10), dpi=80)
fig = plt.figure(figsize=(7, 4), dpi=300)
#fig = plt.figure(figsize=figsize, dpi=300)
ax = fig.gca(projection='3d')

ax.set_xlabel('#Bytes 10^x')
ax.set_ylabel('#Connections 2^x')
ax.set_zlabel('Cycles')

ax.set_zlim(0,600)

#ax.scatter(X, Y, Z)
ax.plot_wireframe(X,Y,Z)
plt.title("Cost of state map by number of connections")
plt.savefig('output/denseMap.pdf',bbox_inches='tight')
plt.show()

#plt.plot(steps, denseMapMean)
#plt.ylim([0,500])
#plt.xlabel('#Connections')
#plt.ylabel('Cycles')
#plt.title("Cost of state map by number of connections")
#plt.show()
#
#dataDense